In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# 加载 MNIST 数据集
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 对数据进行预处理
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# 定义模型
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

# 编译模型
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 训练模型
model.fit(x_train, y_train, epochs=1)

# 评估模型
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('Test accuracy:', test_acc)

model.save('mnist.keras')


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.load_model('mnist.keras')

In [ ]:
# model.summary()

from tensorflow.keras.utils import plot_model

plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
import onnx
import tf2onnx

input_signature = [model.layers[0].input.type_spec]
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature, opset=13)
onnx.save(onnx_model, "model.onnx")

In [48]:
# x_test[0], y_test[0]
y_test[0]
# x_test[0].shape

# type(x_test[0])

# x_test[0].flatten().shape


7

In [53]:
import onnx
# import onnxruntime_cpp

# 加载ONNX模型
model = onnx.load('model.onnx')

# 创建ONNX Quantized模型
quantized_model = onnx.quantize_model(model, onnxruntime_cpp.OrtQuantizationMode.OQM_ONNX_DEFAULT)

# 保存ONNX Quantized模型
quantized_model.save('model.quant.onnx')

AttributeError: module 'onnx' has no attribute 'quantize_model'

In [ ]:
import numpy as np
import struct

tx = x_test[0].flatten().tolist()
tx

# with open('tx.txt', 'w') as f:
#     f.write(struct.pack('i', len(tx)))

#     for value in tx:
#         f.write(struct.pack('f', value))


